### has more interesting about 空資料

# Accumulators Variables

In [237]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("My App") #local ==> to local 執行續
sc = SparkContext(conf = conf)
fileRDD = sc.textFile("/home/jordan/Desktop/callsign_tbl_sorted")

In [241]:
#build a accumulator[Int],and init to 0
blankLines = sc.accumulator(0)

In [242]:
blankLines

Accumulator<id=40, value=0>

In [243]:
def extractCallSigns(line):
    global blankLines
    if (line==""):
        blankLines += 1
    return line.split(" ")
        

In [244]:
callSigns = fileRDD.flatMap(extractCallSigns) #flatMap() Lazy操作

In [245]:
callSigns.saveAsTextFile("/home/jordan/Desktop"+"/callsigns")

In [246]:
print "Blank lines: %d"%blankLines.value

Blank lines: 0


In [236]:
sc.stop()

### 在工作節點不能讀取累加器的value(),=>累加器是write-only

### 錯誤累加器

In [247]:
validSignCount = sc.accumulator(0)
invalidSignCount = sc.accumulator(0)
import re

In [248]:
def validataSign(sign):
    global validSignCount, invalidSignCount
    if re.match(r"\A\d?[a-zA-Z]{1,2}\d{1,4}[a-zA-Z]{1-3}\Z", sign):
        validSignCount += 1
        return True
    else:
        invalidSignCount
        return False
    

In [249]:
#对与每个呼号的联系次数进行计数
validSigns = callSigns.filter(validateSign)
contactCounts = validSigns.map(lambda sign: (sign, 1)).reduceByKey((lambda x, y: x + y))

In [250]:
#強迫評估已產生計數值
contactCounts.count()

0

In [251]:
invalidSignCount.value
type(validSignCount.value)

int

In [252]:
#強迫評估已產生計數值

contactCounts.count()
if invalidSignCount.value < 0.1 * validSignCount.value:
    contactCounts.saveAsTextFile(outputDir + "/contactCount")
else:
    print ("Too many errors %d in %d" %
           (invalidSignCount.value, validSignCount.value))

Too many errors 0 in 0


### 根據叢集的情況,同樣的資料可能會進行多次的訪問 =>在行動中發生時執行累加器,所以Spark在每個子工作中對累加器的更新只會有一次,=>在轉換操作中使用累加器實務上只能用來除錯。

In [158]:
sc.stop()

# Broadcast Variables

### 允與程式有效率的傳送大型且 read 的數值到所有運算節點進行一次,或是多次的操作

In [253]:
def loadCallSignTable():
    f = open("/home/jordan/Desktop/callsign_tbl_sorted", "r")
    return f.readlines()
def lookupCountry(sign, prefixes):
    pos = bisect.bisect_left(prefixes, sign)
    return prefixes[pos].split(",")[1]

In [254]:
 #在Python中使用广播变量来查询国家
 #查询RDD contactCounts中的呼号的对应位置，将呼号前缀读取为国家前缀来进行查询

signPrefixes = sc.broadcast(loadCallSignTable())
  
def processSignCount(sign_count, signPrefixes):
    country = lookupCountry(sign_count[0], signPrefixes.value)
    count = sign_count[1]
    return (country, count)
 
countryContactCounts = (contactCounts
                        .map(lambda signCount: processSignCount(signCount, signPrefixes))
                        .reduceByKey((lambda x, y: x + y)))

In [263]:
countryContactCounts.count()

0

### pre-partition=>避免為每份分割重複進行工作

### Python 共享連線池

In [258]:
def processCallSigns(signs):
    """Lookup call signs using a connection pool"""
    # Create a connection pool
    http = urllib3.PoolManager()
    # the URL associated with each call sign record
    urls = map(lambda x: "http://73s.com/qsos/%s.json" % x, signs)
    # create the requests (non-blocking)
    requests = map(lambda x: (x, http.request('GET', x)), urls)
    # fetch the results
    result = map(lambda x: (x[0], json.loads(x[1].data)), requests)
    # remove any empty results and return
    return filter(lambda x: x[1] is not None, result)


def fetchCallSigns(input):
    """Fetch call signs"""
    return input.mapPartitions(lambda callSigns: processCallSigns(callSigns))

contactsContactList = fetchCallSigns(validSigns)

### 數值RDD = 字串RDD.map(lambda string: float(String))

In [265]:

#stats(StatsCounter物件) = 數值RDD.stats()
stats.count()
stats.mean()
stats.sum()
stats.max()
stats.min()
stats.variance()
stats.sampleVariance()
stats.stdev()
stats.sampleStdev()


'\nstats(StatsCounter\xe7\x89\xa9\xe4\xbb\xb6) = \xe6\x95\xb8\xe5\x80\xbcRDD.stats()\nstats.count()\nstats.mean()\nstats.sum()\nstats.max()\nstats.min()\nstats.variance()\nstats.sampleVariance()\nstats.stdev()\nstats.sampleStdev()\n'